## Fulfilling Natural Language Requests with Azure AI Agent Service

This notebook walks through a simple example of answering natural language requests using multiple agents.

Each agent is small, with one specific purpose. The agents cooperate to provide information in the form of contextual data added to a message thread; this data is used to answer the user query with a structured data (JSON) response.

The four agents we'll create and use are:

- **User Info Agent**: provides a lookup interface for user information (actual data is mocked)
- **City Info Agent**: provides tourist information about cities, sourced from uploaded text files
- **Weather Info Agent**: provides a lookup interface for weather information for a given city (actual data is mocked)
- **Summary Agent**: combines the information from the other agents to provide a structured answer to the user query

The steps in the flow below are:

1. Register each agent with AI Agent Service
1. Start a new agent thread
1. Add the user query as the first message in the thread
1. Ask the user info agent to lookup the user by name and add the user record to the message thread
1. Ask the city info agent to add tourist information about the user's preferred city to the message thread
1. Ask the weather info agent to lookup the weather for the user's preferred city and add that information to the message thread
1. Ask the Summary Agent to summarize the thread and provide a structured answer

_Note this agent flow is deterministic, in that we explicitly invite each agent to the thread, in order. More advanced scenarios can use an orchestrator agent to manage the flow of information and control the order in which agents are invoked; this advanced behavior is not needed for many scenarios, including this one._

_Be conservative with choice of deterministic vs. non-deterministic orchestration._

In [ ]:
user_query = """
Suggest a good tourist activity for Sally, and let her know if she'll need a jacket.
"""

In [ ]:
# setup Python imports

import json
from dotenv import dotenv_values
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet, FileSearchTool, FilePurpose
from azure.identity import DefaultAzureCredential

In [ ]:
# read environment variables... be sure to create .env from .env.example

env = dotenv_values(".env")

In [ ]:
# initialize the AI Agent Service connection

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=env["FOUNDRY_CONN"]
)

In [ ]:
# create and register the city info agent with AI Agent Service

ny_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/new_york.txt', purpose=FilePurpose.AGENTS)
london_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/london.txt', purpose=FilePurpose.AGENTS)
tokyo_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/tokyo.txt', purpose=FilePurpose.AGENTS)

vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[ny_file.id, london_file.id, tokyo_file.id], name="city_info_vector_store")

city_info_tool = FileSearchTool(vector_store_ids=[vector_store.id])

toolset = ToolSet()
toolset.add(city_info_tool)

instructions = """
You are a tourist info bot. Use the provided tools and resources to answer questions about cities and things to do.
Do not answer any other questions.
"""

city_info_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="city-info-agent",
    instructions=instructions,
    toolset=toolset
)

In [ ]:
# create and register the user info agent with AI Agent Service

def get_user_info_by_name(name) -> str:
    return json.dumps({
        'user_id': 17,
        'name': name,
        'preferred_city': 'Tokyo'
    })

functions = FunctionTool([get_user_info_by_name])
toolset = ToolSet()
toolset.add(functions)

instructions = """
You are a user info bot.
Use the provided tools to lookup info about users by name.
Return only the JSON tool output.
Do not answer any other questions.
"""

user_info_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="user-info-agent",
    instructions=instructions,
    toolset=toolset,
    temperature=0.1,
    top_p=0.1,
    response_format={ "type": "json_object" }
)

In [ ]:
# create and register the weather info agent with AI Agent Service

def fetch_weather_for_city(city: str) -> str:
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 12°C"}
    weather = mock_weather_data.get(city, "Weather data not available for this city.")
    return json.dumps({
        "city": city,
        "weather": weather
    })

functions = FunctionTool([fetch_weather_for_city])
weather_toolset = ToolSet()
weather_toolset.add(functions)

instructions = """
You are a weather info bot.
Use the provided tools to get weather for the city.
Return only the JSON tool output.
Do not answer any other questions.
"""

weather_info_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="weather-info-agent",
    instructions=instructions,
    toolset=weather_toolset,
    temperature=0.1,
    top_p=0.1,
    response_format={ "type": "json_object" }
)

In [ ]:
# create and register the summary agent with AI Agent Service

instructions = """
You are a summary bot.
Use information from the message thread to produce output in the following format.
Do not do anything else.

Expected JSON format:

{
  "user_id": <id>,
  "user_name": <name>,
  "preferred_city": <city>,
  "suggested_activity": <activity>,
  "jacket_needed": <boolean>
}
"""

summary_agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="summary-agent",
    instructions=instructions,
    temperature=0.1,
    top_p=0.1
)

In [ ]:
# start a new agent thread, add the user query as the first message

thread = project_client.agents.create_thread()

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_query
)

In [ ]:
# add the user info agent to the thread, it will provide context according to its instructions

user_info_run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=user_info_agent.id)

if user_info_run.status == "failed":
    print(f"Run failed: {user_info_run.last_error}")


In [ ]:
# add the city info agent to the thread, it will provide context according to its instructions
#  note the city info added as context is based on the user's preferred city, as returned by the user info agent

city_info_run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=city_info_agent.id)

if city_info_run.status == "failed":
    print(f"Run failed: {city_info_run.last_error}")


In [ ]:
# add the weather info agent to the thread, it will provide context according to its instructions
#  note the city used is the one returned by the user info agent

weather_run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=weather_info_agent.id)

if weather_run.status == "failed":
    print(f"Run failed: {weather_run.last_error}")

In [ ]:
# add the summary agent to the thread, it will produce the final output according to its instructions
#  note the summary agent will have access to all previous messages in the thread

summary_run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=summary_agent.id)

if summary_run.status == "failed":
    print(f"Run failed: {summary_run.last_error}")

In [ ]:
# retrieve the messages in the thread

messages = project_client.agents.list_messages(thread_id=thread.id)

In [ ]:
# view the full message thread

print(json.dumps(messages.as_dict(), indent=2))

In [ ]:
# view the summary agent's output

print(messages.as_dict()["data"][0]["content"][0]["text"]["value"])